In [69]:
# 生成用于分类的数据，并为每个数据点打标签
import numpy as np
import matplotlib.pyplot as plt
import math
import random
import copy

def tag_entry(x,y):
    if x**2+y**2<1:
        tag=0
    else:
        tag=1
    return tag

def create_data(Number_of_Data):
    entry_list=[]
    for i in range (Number_of_Data):
        x=random.uniform(-2,2)
        y=random.uniform(-2,2)
        tag=tag_entry(x,y)
        entry_list.append([x,y,tag])
    return np.array(entry_list)


In [70]:
#可视化
def plot_data(data,title):
    colors=[]
    for i in data[:,2]:
        if i==0:
            colors.append("orange")
        else:
            colors.append("blue")
    plt.scatter(data[:,0],data[:,1],c=colors)
    plt.title(title)
    plt.show()


In [71]:
def activation_ReLU(inputs):
    return np.maximum(0,inputs)

def activation_softmax(inputs):
    max_value=np.max(inputs,axis=1,keepdims=True)
    slided_inputs=inputs-max_value
    exp_values=np.exp(slided_inputs)
    norm_base=np.sum(exp_values,axis=1,keepdims=True)
    norm_values=exp_values/norm_base
    return norm_values

In [72]:
class Layer:

    def __init__(self,n_inputs,n_neurons):
        self.weights=np.random.randn(n_inputs,n_neurons)  
        self.biases=np.random.randn(n_neurons)  

    def forward(self,inputs):
        self.sum=np.dot(inputs,self.weights)+self.biases  
        return self.sum  


In [73]:

class Network:
    def __init__(self,network_shape):
        self.shape=network_shape
        self.layers=[]
        for i in range(len(network_shape)-1):
            layer=Layer(network_shape[i],network_shape[i+1])
            self.layers.append(layer)
    #前馈运算
    def network_forward(self,inputs):
        outputs=[inputs]
        for i in range(len(self.layers)):
            layer_output=self.layers[i].forward(outputs[i])
            if i==len(self.layers)-1:
                layer_output=activation_softmax(layer_output)
            else:
                layer_output=activation_ReLU(layer_output)
            outputs.append(layer_output)
        return outputs

In [74]:
#分类函数
def classify(probabilities):
    classification=np.rint(probabilities[:,1])
    return classification

In [75]:
def precise_loss_function(predicted,real):

    # 创建真实标签的one-hot编码矩阵
    real_matrix=np.zeros((len(real),2))  # 使用是len(real)
    real_matrix[:,1]=real.flatten()  # 使用flatten()将(1,5)转换为(5,)，赋值给第二列（正类）
    real_matrix[:,0]=1-real  # 第一列为负类（1-正类标签）
    print("真实标签补全矩阵:\n",real_matrix)
    
    # 计算预测值与真实标签的点积，沿axis=1求和
    product=np.sum(predicted*real_matrix,axis=1)
    
    # 返回损失值：1减去点积
    return 1-product


举例说明：
假设有一个样本，真实标签是1（正类），预测概率是[0.8, 0.2]：
转换为one-hot：target = [0, 1]
检查预测：点积 = 0×0.8 + 1×0.2 = 0.2 < 0.5（预测错误）

计算demands：
原始误差：[0, 1] - 0.5 = [-0.5, 0.5]
放大误差：[-0.5, 0.5] × 2 = [-1, 1]
结果：返回[-1, 1]，意思是"强烈减少负类输出，强烈增加正类输出"

如果预测是[0.2, 0.8]（正确），点积=0.8>0.5，则返回[0, 0]（无需调整）。
放大的意义：将-0.5到0.5的微弱信号放大到-1到1，让权重调整更显著

In [76]:
def get_final_layer_preAct_demands(predicted_value,target_value):
    """
    计算最终层激活前的需求值（demands），用于反向传播的梯度计算
    
    参数:
    predicted_value: 预测概率值，形状为 (m, 2)，每行为一个样本的[负类概率, 正类概率]
    target_value: 真实标签，形状为 (m,)，值为0或1
    
    返回:
    target: 梯度信号向量，形状为 (m, 2)
            - 如果预测正确：返回 [0, 0]（无需调整）
            - 如果预测错误：返回放大的误差向量，指导权重如何调整
    """
    # 创建one-hot编码矩阵
    target=np.zeros((len(target_value),2))
    target[:,1]=target_value  # 第二列为正类标签
    target[:,0]=1-target_value  # 第一列为负类标签（1-正类）
    
    # 遍历每个样本
    for i in range(len(predicted_value)):
        # 计算预测值与真实标签的点积，判断预测是否正确
        if np.dot(target[i],predicted_value[i])>0.5:
            # 预测正确：点积>0.5，设置为零向量（无需调整权重）
            target[i]=np.array([0,0])
        else:
            # 预测错误：计算放大的误差向量
            # (target-0.5)*2 将-0.5~0.5的误差放大到-1~1，增强梯度信号
            target[i]=(target[i]-0.5)*2
    return target
    

In [77]:
# 测试需求函数
# 修复：np.array() 需要嵌套列表来创建二维数组
predicted_value = np.array([[0., 1],
                            [0.5, 0.5],
                            [0.2, 0.8], 
                            [0.7, 0.3],
                            [0.9, 0.1]])

target_value = np.array([1, 0, 1, 0, 1])

print("预测值 (predicted_value):")
print(predicted_value)
print("\n真实标签 (target_value):")
print(target_value)
print("\n需求值:")
print(get_final_layer_preAct_demands(predicted_value,target_value))


预测值 (predicted_value):
[[0.  1. ]
 [0.5 0.5]
 [0.2 0.8]
 [0.7 0.3]
 [0.9 0.1]]

真实标签 (target_value):
[1 0 1 0 1]

需求值:
[[ 0.  0.]
 [ 1. -1.]
 [ 0.  0.]
 [ 0.  0.]
 [-1.  1.]]


浅拷贝（Shallow Copy）：
只复制对象的第一层结构
嵌套对象仍然是引用，修改嵌套内容会影响原对象
使用：copy.copy() 或 list.copy()

深拷贝（Deep Copy）：
递归复制所有层级的对象
完全独立的副本，修改任何层级都不会影响原对象
使用：copy.deepcopy()

对于NumPy数组：
array.copy() 通常就足够了（NumPy数组元素是基本数据类型）
copy.deepcopy() 在这里有些过度，但确保了完全隔离

第一层结构指对象本身的直接属性，不包括嵌套对象的内容。
例子：
第一层：列表本身 [_, _]
第二层：嵌套的子列表 [1, 2] 和 [3, 4]

浅拷贝结果：
复制了外层列表结构（创建新的列表对象）
但子列表仍然是引用，shallow[0] 和 original[0] 指向同一个 [1, 2]
修改 shallow[0][0] = 99 会同时改变 original[0][0]

简单说： 只复制"容器"，不复制"容器里的东西"

In [ ]:
Number_of_Data=5
data=create_data(Number_of_Data)
print("原数据data:\n",data)
inputs=data[:,0:2]
# 使用 copy.deepcopy() 创建目标标签的深拷贝
# 原因：如果直接使用 tagets = data[:,2]，那么 tagets 只是 data[:,2] 的一个引用
# 当后续代码修改 tagets 时，原始的 data 数组也会被修改，这可能导致：
# 1. 数据污染：原始训练数据被意外修改
# 2. 调试困难：多次运行代码时结果不一致
# 3. 内存共享问题：多个变量指向同一内存地址
# 使用深拷贝确保 tagets 是独立的副本，保护原始数据不被修改
tagets=copy.deepcopy(data[:,2])

net=Network([2,3,4,5,2])
outputs=net.network_forward(inputs)
classification=classify(outputs[-1])

loss=precise_loss_function(outputs[-1],tagets)
print("\nloss:\n",loss)
demands=get_final_layer_preAct_demands(outputs[-1],tagets)
print("\ndemands:\n",demands)

原数据data:
 [[ 0.95766849 -0.2148687   0.        ]
 [ 1.23720219 -1.32012822  1.        ]
 [ 0.71617953  1.18114183  1.        ]
 [ 1.68150705  0.58082246  1.        ]
 [ 1.79711348 -1.25986425  1.        ]]
真实标签补全矩阵:
 [[1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]]

loss:
 [0.05635063 0.9557253  0.8461802  0.84862233 0.9557253 ]

demands:
 [[ 0.  0.]
 [-1.  1.]
 [-1.  1.]
 [-1.  1.]
 [-1.  1.]]
